In [1]:
import numpy as np
from typing import List

# 参考 pymatgen 的 pymatgen/analysis/elasticity/elastic.py 中的 ElasticTensor 类
class ElasticProperty:
    def __init__(self, elastic_tensor: np.ndarray | List[List[float]]):
        self.elastic_tensor = np.array(elastic_tensor)

    @property
    def stiffness_tensor(self) -> np.ndarray:
        return np.linalg.inv(self.elastic_tensor)

    @property
    def k_voigt(self) -> float:
        # 1 * 对角线元素 + 2 * 对角线向上元素和 / 9 等价于 矩阵平均值
        return self.elastic_tensor[:3, :3].mean()

    @property
    def k_reuss(self) -> float:
        return 1 / self.stiffness_tensor[:3, :3].sum()

    @property
    def k_vrh(self) -> float:
        return 0.5 * (self.k_voigt + self.k_reuss)

    @property
    def g_voigt(self) -> float:
        return (
            self.elastic_tensor[:3, :3].trace()
            - np.triu(self.elastic_tensor[:3, :3], k=1).sum()
            + 3 * self.elastic_tensor[3:, 3:].trace()
        ) / 15

    @property
    def g_reuss(self) -> float:
        return 15 / (
            4 * self.stiffness_tensor[:3, :3].trace()
            - 4 * np.triu(self.stiffness_tensor[:3, :3], k=1).sum()
            + 3 * self.stiffness_tensor[3:, 3:].trace()
        )

    @property
    def g_vrh(self) -> float:
        return 0.5 * (self.g_voigt + self.g_reuss)

    @property
    def possion_ratio(self) -> float:
        return 0.5 * (3 * self.k_vrh - 2 * self.g_vrh) / (3 * self.k_vrh + self.g_vrh)

    @property
    def property_dict(self):
        props = (
            "k_voigt",
            "k_reuss",
            "k_vrh",
            "g_voigt",
            "g_reuss",
            "g_vrh",
            "possion_ratio",
        )

        return {prop: getattr(self, prop) for prop in props}

In [2]:
# 立方晶系
cubic_matrix = np.array(
    [
        [1050.640, 126.640, 126.640, 0.000, 0.000, 0.000],
        [126.640, 1050.640, 126.640, 0.000, 0.000, 0.000],
        [126.640, 126.640, 1050.640, 0.000, 0.000, 0.000],
        [0.000, 0.000, 0.000, 559.861, 0.000, 0.000],
        [0.000, 0.000, 0.000, 0.000, 559.861, 0.000],
        [0.000, 0.000, 0.000, 0.000, 0.000, 559.861],
    ]
)

elsp = ElasticProperty(cubic_matrix)
elsp.property_dict

{'k_voigt': 434.64000000000004,
 'k_reuss': 434.64000000000004,
 'k_vrh': 434.64000000000004,
 'g_voigt': 520.7166,
 'g_reuss': 516.1302450950266,
 'g_vrh': 518.4234225475133,
 'possion_ratio': 0.07327739426074346}

In [3]:
# 六方晶系
hexagonal_matrix = np.array(
    [
        [297.85, 126.9, 104.5, 0.0, 0.0, 0.0],
        [126.9, 297.85, 104.5, 0.0, 0.0, 0.0],
        [104.5, 104.5, 286.9, 0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0, 59.225, 0.0, 0.0],
        [0.0, 0.0, 0.0, 0.0, 59.225, 0.0],
        [0.0, 0.0, 0.0, 0.0, 0.0, 85.475],
    ]
)

elsp = ElasticProperty(hexagonal_matrix)
elsp.property_dict

{'k_voigt': 172.7111111111111,
 'k_reuss': 172.2853759366118,
 'k_vrh': 172.49824352386145,
 'g_voigt': 77.23166666666667,
 'g_reuss': 74.04254440603603,
 'g_vrh': 75.63710553635136,
 'possion_ratio': 0.3087176384781328}